# 1. Establish communication with the card

## 1.1. Find the smartcard reader

In [1]:
from applets.helpers import get_connection
from applets.memorycard import MemoryCard

In [2]:
connection = get_connection()
if connection is None:
    print("Failed to open a connection. No cardreaders?")

## 1.2. Power on the card and get `ATR`
`ATR` - Answer To Reset, bytes retured by the card when it is powered on.<br>
It helps to determine communication protocol and frequency we should choose.

In [3]:
atr=bytes(connection.getATR())
print(atr.hex())

3bdc18ff8191fe1fc38073c821136605036351000250


## 1.3. Select the applet by AID
JavaCard can have multiple applets installed. They are like very isolated "apps".<br>
They can't talk to each other and only one can be active at any time.

In [4]:
app = MemoryCard(connection)
app.select()

b''

In [5]:
# get random data without secure communication
app.get_random().hex()

'8fe6cc1a5b7129d458b17adb887453794606057ae1dc5af9141104fc205dc0c7'

# 2. Establish secure communication

At the moment we use a homebrew version, I need to understand and implement Noise in the near future.

[Noise protocol](https://noiseprotocol.org/noise.html) should be with the following primitives:
- secp256k1 as signature scheme
- AES CBC with random IV as a symmetric cipher
- SHA256 as a hash function, HMAC_SHA256 as authentication code

AES CBC because we have hardware implementation on the card, so we can use the Key class that is more secure than byte array. HMAC sucks here though...

Generally a shared secret is split to 2 different secrets for encryption and decryption. This allows us to use AES in counter mode where the counter is the same for both host and the card - they use different keys so it's ok.

Description of the current protocol:
1. Get the static public key from the card. If it is already known - proceed with step 2
2. Generate a random key pair and send corresponding pubkey in uncompressed form to the card: `<data:pub_uncompressed>`
3. Card will return its fresh session public key, hmac of this public key using shared secret and signature: `<data:session_pubkey_card><hmac(dec,data)><sig(static_pubkey_card,data|hmac)>`
4. Verify the signature against card's pubkey
5. Calculate a pair of shared secrets from `enc=SHA256('host'|ECDH(d,session_pubkey_card))`, `dec=SHA256('card'|ECDH(d,session_pubkey_card)`
6. Verify that `hmac==HMAC_SHA256(dec,session_pubkey_card)`
7. Set counter to zero `iv=0`, use `dec` for decryption, `enc` for enctyption. Increase the counter on every request, reestablish communication if the counter reaches max value (16 bytes, shouldn't happen in most cases)
8. All secure messages now should be in the form `<ct:aes(enc,iv,command|data)><hmac(enc,iv|ct)>`, `iv` is not transmitted but increased for every new command and used in HMAC. Returned data will be of the same form but using the `dec` key. `iv` will be the same.

Length of the secure message is always mod 16 (AES blocks + 32-byte HMAC), except when it has maximal length - 255 bytes. This is a special case when we are hitting the limit of single-byte length, but it is still very usable for mnemonic storage, so we just drop the last byte of HMAC - doesn't decrease security much, but gives us 16 extra bytes of data!

## Why we drop last byte of HMAC

Maximum size of a simple APDU is 255 bytes. Extended can be larger though, but it makes everything complicated. Maximum size of the mnemonic is `9*24-1=215` bytes plus length and command and rounding to `16` bytes, plus `32`-byte hmac would mean `256` bytes length... We can send at max `255` bytes, so we decided to always drop last byte in HMAC.

Later we should add support of extended APDU commands, but for now it's ok.


In [6]:
app.establish_secure_channel() # mode="se" and mode="ee" available, "se" by default

In [7]:
app.secure_request(b'ping')

b'qjoh'

In [8]:
mnemonic = b'universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe urge'
app.secure_request(mnemonic)

b'vojwfstf!vojwfstf!vojwfstf!vojwfstf!vojwfstf!vojwfstf!vojwfstf!vojwfstf!vojwfstf!vojwfstf!vojwfstf!vojwfstf!vojwfstf!vojwfstf!vojwfstf!vojwfstf!vojwfstf!vojwfstf!vojwfstf!vojwfstf!vojwfstf!vojwfstf!vojwfstf!vshf'

In [9]:
connection.disconnect()